In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

### Collaborative Filtering

### 데이터 준비

In [2]:
user_matrix=pd.read_csv("user_matrix.csv", index_col=0)
user_matrix=user_matrix[['movie_id', 'user_id','rate']]

idx=user_matrix[user_matrix['rate']=='보고싶어요'].index
user_matrix=user_matrix.drop(idx)
idx=user_matrix[user_matrix['rate']=='보는중'].index
user_matrix=user_matrix.drop(idx)

user_matrix['rate']=user_matrix['rate'].map(lambda x:float(x))
user_matrix.head()

,movie_id,user_id,rate
0,0,6151,4.5
1,0,5468,5.0
2,0,2160,4.0
3,0,4076,5.0
4,0,5531,5.0


In [3]:
user_matrix_10 = user_matrix.groupby('user_id').filter(lambda x : len(x)> 10).copy()

In [4]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.datasets import amazon_clothing
from cornac.data import Reader

In [5]:
user_data=[tuple(x) for x in user_matrix.values]
user_data_10=[tuple(x) for x in user_matrix_10.values]

In [6]:
user_data_10

[(0.0, 2160.0, 4.0),
 (0.0, 544.0, 3.5),
 (0.0, 3238.0, 4.5),
 (0.0, 5144.0, 4.0),
 (0.0, 2126.0, 5.0),
 (0.0, 6026.0, 3.0),
 (0.0, 1158.0, 4.0),
 (0.0, 1350.0, 4.0),
 (0.0, 472.0, 3.5),
 (0.0, 4074.0, 4.0),
 (0.0, 3602.0, 3.5),
 (0.0, 2219.0, 4.0),
 (0.0, 3180.0, 3.5),
 (0.0, 1254.0, 4.5),
 (0.0, 1962.0, 3.0),
 (0.0, 4105.0, 3.5),
 (0.0, 3683.0, 4.0),
 (0.0, 2018.0, 3.5),
 (0.0, 2049.0, 4.0),
 (0.0, 2203.0, 4.0),
 (0.0, 287.0, 3.0),
 (0.0, 6035.0, 3.5),
 (0.0, 4072.0, 3.0),
 (0.0, 1809.0, 4.0),
 (1.0, 4632.0, 3.5),
 (1.0, 5166.0, 3.0),
 (1.0, 2752.0, 3.5),
 (1.0, 1952.0, 2.5),
 (1.0, 234.0, 3.0),
 (1.0, 1527.0, 3.0),
 (1.0, 2464.0, 3.5),
 (1.0, 2396.0, 4.0),
 (1.0, 3346.0, 4.0),
 (1.0, 1502.0, 3.0),
 (1.0, 230.0, 4.0),
 (1.0, 4340.0, 4.0),
 (1.0, 2018.0, 2.5),
 (1.0, 3474.0, 1.5),
 (1.0, 619.0, 2.5),
 (1.0, 454.0, 3.0),
 (1.0, 2899.0, 5.0),
 (1.0, 1891.0, 4.0),
 (1.0, 4618.0, 3.0),
 (1.0, 3150.0, 3.5),
 (1.0, 857.0, 4.0),
 (1.0, 779.0, 2.5),
 (1.0, 4532.0, 2.0),
 (1.0, 627.0, 3.0),
 (

In [7]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=user_data_10,
    test_size=0.2,
    rating_threshold=1.0,
    seed=123,
    exclude_unknowns=True,
    verbose=True,
)

# Instantiate the recommender models to be compared
neumf1 = cornac.models.NeuMF(
    num_factors=8,
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)

# Instantiate evaluation metrics
ndcg_50 = cornac.metrics.NDCG(k=50)
rec_50 = cornac.metrics.Recall(k=50)
pre_50 = cornac.metrics.Precision(k=50)
f1_50 = cornac.metrics.FMeasure(k=50)

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[neumf1],
    metrics=[ndcg_50, rec_50, pre_50, f1_50],
).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 669
Number of items = 508
Number of ratings = 15068
Max rating = 5.0
Min rating = 0.5
Global mean = 3.4
---
Test data:
Number of users = 662
Number of items = 495
Number of ratings = 3772
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 669
Total items = 508

[NeuMF] Training started!


c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 24 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 1 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a futu


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.





[NeuMF] Evaluation started!




TEST:
...
      |  F1@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
----- + ------ + ------- + ------------ + --------- + --------- + --------
NeuMF | 0.0672 |  0.1800 |       0.0382 |    0.3371 |  413.9677 |   1.6412



In [81]:
user_1_ncf=pd.DataFrame(neumf1.rank(1)).T
user_1_ncf

,0,1
0,31.0,0.008509
1,2.0,0.020827
2,118.0,0.157921
3,68.0,0.021750
4,15.0,0.082053
...,...,...
503,431.0,0.009089
504,445.0,0.008397
505,476.0,0.009716
506,490.0,0.009067


In [ ]:
정규화 방법 설명하고, 비율만 하는 방법 말고 클러스터 정규화 같은 것도 필요할지

### Content-based Filtering

### 데이터 준비

In [8]:
adj_cluster=pd.read_csv("adj_cluster.csv", index_col=0)
adj_cluster.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,spiderman,dirty,much,know,sentimental,sudden,later,charming,alltime,koreanstyle,...,scared,central,necessary,life,religious,limited,male,heartbreaking,fourth,middle
1,NaN,NaN,many,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,main,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,great,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#words_num: 클러스터 별 단어 개수
columns=len(adj_cluster.columns)
max_len=len(adj_cluster)
words_num=[max_len-adj_cluster[str(index)].isnull().sum() for index in range(columns)]

In [12]:
#아이템/유저 별 단어 개수
movie_adj_matrix.iloc[1,:].sum()

91.0

In [11]:
movie_adj_matrix=pd.read_csv('movie_adj_matrix.csv', index_col=0)
movie_adj_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,41.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def minmax(rowmin, rowmax, xi):
    return (xi-rowmin)/(rowmax-rowmin)

In [14]:
for i in range(len(movie_adj_matrix.columns)):
    row_sum=movie_adj_matrix.iloc[i,:].sum()
    movie_adj_matrix[str(i)]=movie_adj_matrix[str(i)].map(lambda x:x/(words_num[i]))
    rowmax=max(movie_adj_matrix.iloc[i,:])
    rowmin=min(movie_adj_matrix.iloc[i,:])
    for j in range(100):
        movie_adj_matrix.iloc[i,j]=minmax(rowmin, rowmax, movie_adj_matrix.iloc[i,j])

In [40]:
movie_adj_matrix

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.004975,0.0,0.02439,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.024390
1,0.0,0.0,0.004975,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.025641
2,0.0,0.0,0.009524,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.006160,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.095238,0.047619,0.0,0.0,0.0,0.0,0.0,0.047619
4,0.0,0.0,0.014340,0.0,0.00000,0.0,0.0,0.0,0.058824,0.0,...,0.0,0.0,0.000000,0.058824,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,0.0,0.0,0.203980,0.0,0.00000,0.0,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
666,0.0,0.0,0.109453,0.0,0.00000,0.0,0.0,4.0,0.000000,0.0,...,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000
667,0.0,0.0,0.233831,0.0,1.00000,0.0,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
668,0.0,0.0,0.194030,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [41]:
user_adj_matrix=pd.read_csv('user_adj_matrix.csv', index_col=0)
user_adj_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [42]:
for i in range(len(user_adj_matrix.columns)):
    row_sum=user_adj_matrix.iloc[i,:].sum()
    user_adj_matrix[str(i)]=user_adj_matrix[str(i)].map(lambda x:x/(words_num[i]))
    rowmax=max(user_adj_matrix.iloc[i,:])
    rowmin=min(user_adj_matrix.iloc[i,:])
    for j in range(100):
        if rowmax==0:
            break
        user_adj_matrix.iloc[i,j]=minmax(rowmin, rowmax, user_adj_matrix.iloc[i,j])

In [43]:
user_adj_matrix

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.004975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.009950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.011307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,0.0,0.0,0.004975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6152,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6153,0.0,0.0,0.009950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6154,0.0,0.0,0.014925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [78]:
movie_user_1=movie_adj_matrix
movie_user_1['preference']=[0]*len(movie_user_1)
movie_user_1=movie_user_1.iloc[:508,]

In [79]:
#user 1
for i in range(507):
    movie_user_1.iloc[i,100]=sum(user_adj_matrix.iloc[1,:]*movie_adj_matrix.iloc[i,:])
#     print(sum(user_adj_matrix.iloc[1,:]*movie_adj_matrix.iloc[i,:]))

c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [80]:
movie_user_1

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,preference
0,0.0,0.0,0.004975,0.0,0.02439,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.024390,0.000059
1,0.0,0.0,0.004975,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.025641,0.000044
2,0.0,0.0,0.009524,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000117
3,0.0,0.0,0.006160,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.095238,0.047619,0.0,0.0,0.0,0.0,0.0,0.047619,0.000100
4,0.0,0.0,0.014340,0.0,0.00000,0.0,0.0,0.0,0.058824,0.0,...,0.0,0.000000,0.058824,0.0,0.0,0.0,0.0,0.0,0.000000,0.000141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,0.0,1.0,0.154229,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.001392
504,0.0,0.0,0.199005,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.002448
505,0.0,0.0,0.154229,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.001948
506,0.0,0.0,0.084577,0.0,1.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000699


In [88]:
user_1_ncf.sort_values('result', ascending=False)

,0,1,result
151,296.0,0.058601,2.224491e-04
115,1.0,0.077351,2.181064e-04
105,30.0,0.044600,2.122365e-04
96,181.0,0.060587,1.948521e-04
158,169.0,0.048466,1.890429e-04
...,...,...,...
41,49.0,0.008757,6.075337e-07
0,31.0,0.008509,4.988874e-07
81,368.0,0.015640,3.871300e-07
75,97.0,0.009120,2.257463e-07


In [83]:
user_1_ncf['result']=[0]*len(user_1_ncf)

In [85]:
for i in range(len(user_1_ncf)):
    user_1_ncf.iloc[i,2]=user_1_ncf.iloc[i,1]*movie_user_1.iloc[i,100]

In [55]:
sum(user_adj_matrix.iloc[1,:]*movie_adj_matrix.iloc[0,:])

5.86272013310465e-05

In [31]:
# user_cf['p_id']=[0]*len(user_cf)
user_cf['preference']=[0]*len(user_cf)

In [36]:
user_cf.at[1,'preference']=1

In [ ]:
user_cf

In [19]:
import cornac
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit


K = 50  # number of nearest neighbors


# Load ML-100K dataset
feedback = movielens.load_feedback(variant="100K")

# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=user_data_10, test_size=0.2, exclude_unknowns=True, verbose=True, seed=123
)

# UserKNN methods
user_knn_cosine = cornac.models.UserKNN(k=K, similarity="cosine", name="UserKNN-Cosine")
user_knn_pearson = cornac.models.UserKNN(
    k=K, similarity="pearson", name="UserKNN-Pearson"
)
user_knn_amp = cornac.models.UserKNN(
    k=K, similarity="cosine", amplify=2.0, name="UserKNN-Amplified"
)
user_knn_idf = cornac.models.UserKNN(
    k=K, similarity="cosine", weighting="idf", name="UserKNN-IDF"
)
user_knn_bm25 = cornac.models.UserKNN(
    k=K, similarity="cosine", weighting="bm25", name="UserKNN-BM25"
)
# ItemKNN methods
item_knn_cosine = cornac.models.ItemKNN(k=K, similarity="cosine", name="ItemKNN-Cosine")
item_knn_pearson = cornac.models.ItemKNN(
    k=K, similarity="pearson", name="ItemKNN-Pearson"
)
item_knn_adjusted = cornac.models.ItemKNN(
    k=K, similarity="cosine", mean_centered=True, name="ItemKNN-AdjustedCosine"
)

# Put everything together into an experiment
cornac.Experiment(
    eval_method=ratio_split,
    models=[
        user_knn_cosine,
        user_knn_pearson,
        user_knn_amp,
        user_knn_idf,
        user_knn_bm25,
        item_knn_cosine,
        item_knn_pearson,
        item_knn_adjusted,
    ],
    metrics=[cornac.metrics.RMSE()],
    user_based=True,
).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 669
Number of items = 508
Number of ratings = 15068
Max rating = 5.0
Min rating = 0.5
Global mean = 3.4
---
Test data:
Number of users = 662
Number of items = 495
Number of ratings = 3772
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 669
Total items = 508

[UserKNN-Cosine] Training started!


c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 24 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 1 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)




[UserKNN-Cosine] Evaluation started!




[UserKNN-Pearson] Training started!




[UserKNN-Pearson] Evaluation started!


...



[ItemKNN-Pearson] Training started!




[ItemKNN-Pearson] Evaluation started!




[ItemKNN-AdjustedCosine] Training started!




[ItemKNN-AdjustedCosine] Evaluation started!




TEST:
...
                       |   RMSE | Train (s) | Test (s)
---------------------- + ------ + --------- + --------
UserKNN-Cosine         | 0.6058 |    0.1346 |   1.4617
UserKNN-Pearson        | 0.6370 |    0.1421 |   1.2786
UserKNN-Amplified      | 0.6058 |    0.8317 |   1.3173
UserKNN-IDF            | 0.6062 |    0.1366 |   1.3248
UserKNN-BM25           | 0.6060 |    0.1346 |   1.3056
ItemKNN-Cosine         | 0.7011 |    0.0947 |   1.3574
ItemKNN-Pearson        | 0.7108 |    0.1865 |   1.2936
ItemKNN-AdjustedCosine | 0.6625 |    0.1307 |   1.3000



In [20]:
user_knn_idf.score(user_idx=1)

array([3.770827  , 4.09432958, 3.62831448, 2.66654354, 2.71076991,
       3.89011849, 3.97864726, 3.60349834, 4.18342009, 2.89956895,
       3.58482387, 3.6004619 , 3.58425655, 3.37179816, 3.24161365,
       3.5755642 , 3.60158377, 3.29004869, 3.18406739, 3.80199554,
       3.62148065, 3.57895428, 4.12925059, 4.0508625 , 3.90938915,
       3.57330311, 3.86010928, 3.27580288, 3.39701155, 3.64364341,
       2.90981779, 3.63918893, 3.50206753, 4.36123287, 3.93613333,
       3.98469732, 2.72598699, 3.79593019, 3.88308403, 3.31440927,
       4.0118192 , 4.2134426 , 3.35510434, 3.56562731, 3.76966249,
       3.47058266, 4.73991732, 4.08758019, 3.79798707, 3.79272786,
       3.91364965, 3.41562716, 3.74444961, 4.23014556, 3.43601567,
       3.41639079, 3.69604996, 2.36994719, 3.57033103, 1.4451401 ,
       2.71237959, 4.13000178, 3.62134542, 3.46863138, 3.83225393,
       2.44546811, 2.88615279, 4.41321203, 3.91584045, 3.23530038,
       4.44434768, 3.4099769 , 3.44904574, 3.98506331, 4.31715

### Hybrid Method